# Working with AWS SQS


## Purpose
This notebook is desinged for AWS developers to play around different ways to interact with AWS SQS.


# Setup

## Import Libraries
We import all the required Python libraries.

In [1]:
import boto3
from botocore.exceptions import ClientError
import json
import decimal

In [2]:
boto3.__version__

'1.20.24'

In [3]:
#Determine which credentials(profile) to use embeded in .aws/config file.
boto3 = boto3.session.Session(profile_name='AWS-DEMO')

## Load DynamoDB in us-west-2 region

In [4]:
sqs = boto3.resource('sqs', region_name='us-west-2')

In [5]:
# Create a queue named Demo
queue = sqs.create_queue(QueueName='Demo', Attributes={'DelaySeconds': '5'})

In [6]:
queue.url

'https://us-west-2.queue.amazonaws.com/465968594676/Demo'

In [7]:
# Print out each queue name, which is part of its ARN
for queue in sqs.queues.all():
    print(queue.url)

https://us-west-2.queue.amazonaws.com/465968594676/Demo
https://us-west-2.queue.amazonaws.com/465968594676/DemoQueue
https://us-west-2.queue.amazonaws.com/465968594676/Whatever


In [8]:
# Create a new message
response = queue.send_message(MessageBody='Welcome to Developing on AWS!')
response

{'MD5OfMessageBody': '2aef66df948230da93b11aa26264f5e4',
 'MessageId': '5ffb8fa7-0374-4abf-945d-ddaedecc97cf',
 'ResponseMetadata': {'RequestId': 'e5509747-aeee-54a9-a035-cd2cc92fea3e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e5509747-aeee-54a9-a035-cd2cc92fea3e',
   'date': 'Wed, 26 Jan 2022 15:15:13 GMT',
   'content-type': 'text/xml',
   'content-length': '378'},
  'RetryAttempts': 0}}

In [9]:
response.get('MessageId')

'5ffb8fa7-0374-4abf-945d-ddaedecc97cf'

In [10]:
response.get('MD5OfMessageBody')

'2aef66df948230da93b11aa26264f5e4'

In [11]:
#Messages can also be sent in batches. 
#For example, sending the two messages described above in a single request would look like the following:

response = queue.send_messages(Entries=[
    {
        'Id': '1',
        'MessageBody': 'You are the best students in the world!'
    },
    {
        'Id': '2',
        'MessageBody': 'Hello im boto3',
        'MessageAttributes': {
            'Author': {
                'StringValue': 'Younos',
                'DataType': 'String'
            }
        }
    }
])

# Print out any failures
print(response.get('Failed'))

None


In [25]:
# Process messages by printing out body and optional author name
for message in queue.receive_messages(MessageAttributeNames=['Author']):
    # Get the custom author message attribute if it was set
    author_text = ''
    if message.message_attributes is not None:
        author_name = message.message_attributes.get('Author').get('StringValue')
        if author_name:
            author_text = ' ({0})'.format(author_name)

    # Print out the body and author (if set)
    print('Hello, {0}!{1}'.format(message.body, author_text))

    # Let the queue know that the message is processed
    message.delete()

In [26]:
sqs.meta.client.delete_queue(QueueUrl=queue.url)

{'ResponseMetadata': {'RequestId': '8168daf3-418c-5406-b8fd-809829b16a91',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8168daf3-418c-5406-b8fd-809829b16a91',
   'date': 'Wed, 26 Jan 2022 15:17:17 GMT',
   'content-type': 'text/xml',
   'content-length': '211'},
  'RetryAttempts': 0}}